In [ ]:
import requests

In [ ]:
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/UK/GBP/en-GB/"

params = {"query":"Berlin"}

headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': "7a7b77879fmsh7f229a55585446cp11e799jsna7f7eb740123"}

response = requests.get(url, headers = headers, params = params)
response.json()

In [ ]:
# flights from where to where on what date 

url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/UK/GBP/en-GB/SFO-sky/NYCA-sky/2020-12-12"

params = {"inboundpartialdate":"2020-12-12"}

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "7a7b77879fmsh7f229a55585446cp11e799jsna7f7eb740123"}

response = requests.get(url, headers=headers, params=params)

response.json()

In [ ]:
import pandas as pd 
import json 

In [ ]:
pd.json_normalize(response.json())

In [ ]:
quotes = pd.DataFrame(pd.json_normalize(response.json())["Quotes"][0])
quotes

In [ ]:
flights = pd.DataFrame(pd.DataFrame(pd.json_normalize(response.json())["Quotes"][0])["OutboundLeg"][0])
flights

In [ ]:
carriers=pd.DataFrame(pd.json_normalize(response.json())["Carriers"][0])
carriers

In [ ]:
places= pd.DataFrame(pd.json_normalize(response.json())["Places"][0])
places

In [ ]:
#once we have our pandas we can use the data 

#function 1 - returns the date of the cheapest flight

def lowest_price(origin, destination, start, end):
    flights = flight_days(origin, destination, start, end)
    prices = {date:min([quote["MinPrice"] for quote in flights[date]["Quotes"]]) for date in flights for quote in flights[date]["Quotes"]}
    return min(prices, key = prices.get)


#NOTE The problem with this code is that 
# it just gets the first date as the only one 
# even if there are other days with the same price. 
# Try to fix that in the sky scanner lab (optional)

In [ ]:
# function 2 - function for which if we introduce two dates, 
# an origin and a destination we will get the data from the 
# cheapest flight between these two dates.

#We will start by creating a function that takes two dates
# and return a list of dates between the given ones. 
# Luckily, there is a pandas method for that.

def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format())

In [ ]:
#function 2 continued
# takes the origin, destination and dates as arguments. 
#It will create a dictionary that has every day between the two dates
# and contains a call to the API with all the information about the days


def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    return {date:flight_prices(origin, destination, date) for date in dates}